# Roll damping regression
Development of an empirical regression method to predict roll damping

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from rolldecayestimators.substitute_dynamic_symbols import lambdify,run

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

import signal_lab
from sqlalchemy.inspection import inspect
import seaborn as sns
import docs
from sympy.parsing.sympy_parser import parse_expr
import sympy as sp
from rolldecayestimators import symbols


In [ ]:
data = pd.read_pickle('data.sav')
y_s = pd.read_pickle('y.sav')

In [ ]:
y_key = 'omega0_hat'

fig,ax=plt.subplots()
y_s[y_key].hist(bins=50, ax=ax)
ax.set_title('Historgram: %s' % y_key)

In [ ]:
ks = np.arange(1,10)
degrees = np.arange(1,3)
results = pd.DataFrame()

variance_treshold = VarianceThreshold(0.0001)
standard_scaler = StandardScaler()

y = y_s[y_key]
#X = data[important]
X=data

for k in ks:
    for degree in degrees:
        select_k_best = SelectKBest(k=k, score_func=f_regression)
        
       
        polynomial_features = PolynomialFeatures(degree=degree)
        linear_regression = LinearRegression()
        
        steps=[
            ('polynomial_feature', polynomial_features),
            ('standard_scaler', standard_scaler),
            ('variance_treshold',variance_treshold),
            ('select_k_best',select_k_best),
            ('linear_regression', linear_regression)
        ]
        
        model = Pipeline(steps=steps)
        score = cross_val_score(estimator=model,X=X,y=y,cv=5).mean()
        s = pd.Series()
        s['k'] = k
        s['degree'] = degree
        s['score'] = score
        results = results.append(s, ignore_index=True)
        
        

In [ ]:
results.sort_values(by='score', ascending=False, inplace=True)

In [ ]:
results.head()

In [ ]:
k = int(results.iloc[0]['k'])
degree = int(results.iloc[0]['degree'])

In [ ]:
select_k_best = SelectKBest(k=k, score_func=f_regression)

standard_scaler = StandardScaler()
polynomial_features = PolynomialFeatures(degree=degree)
linear_regression = LinearRegression()

steps=[
    ('polynomial_feature', polynomial_features),
    ('standard_scaler', standard_scaler),
    ('variance_treshold',variance_treshold),
    ('select_k_best',select_k_best),
    #('linear_regression', linear_regression)
]

preprocessor = Pipeline(steps=steps)

steps=[
    ('polynomial_feature', polynomial_features),
    ('standard_scaler', standard_scaler),
    ('variance_treshold',variance_treshold),
    ('select_k_best',select_k_best),
    #('linear_regression', linear_regression)
]

model = linear_regression

In [ ]:
preprocessor.fit(X,y)

In [ ]:
feature_names = np.array(polynomial_features.get_feature_names())
mask_treshold = variance_treshold.get_support()
mask_select_k_best = select_k_best.get_support()

In [ ]:
feature_names_index = pd.Series(feature_names)

In [ ]:
good_index = feature_names_index[mask_treshold][mask_select_k_best]

In [ ]:
good_index

In [ ]:
X2 = polynomial_features.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X2[:,good_index.index], y, test_size=0.20)
linear_regression.fit(X=X_train, y=y_train)

score = linear_regression.score(X=X_test, y=y_test)
score

In [ ]:
sympy_symbols = {key:getattr(symbols,key) for key in X.columns}
parameters = [sympy_symbols[key] for key in X.columns]

In [ ]:
def get_feature_equations(polynomial_feature:PolynomialFeatures, columns):
    
    feature_names = np.array(polynomial_feature.get_feature_names())
    parameters = [sympy_symbols[key] for key in columns]
    
    xs={'x%i'%i:name for i,name in enumerate(X.columns)}
    xs_sympy={sp.Symbol(key):sympy_symbols[value] for key,value in xs.items()}
        
    feature_eqs = [1.0,]

    for feature in feature_names[1:]:
        s_eq = feature
    
        s_eq = s_eq.replace(' ','*')    
        s_eq = s_eq.replace('^','**')    
    
        sympy_eq = parse_expr(s_eq)
        sympy_eq = sympy_eq.subs(xs_sympy)
        feature_eqs.append(sympy_eq)
        
    return np.array(feature_eqs)
    

In [ ]:
feture_equations = get_feature_equations(polynomial_feature=polynomial_features, columns=X.columns)

In [ ]:
good_feature_equations = feture_equations[good_index.index]

In [ ]:
linear_regression.coef_

In [ ]:
rhs = 0 
for good_feature_equation,coeff in zip(good_feature_equations,linear_regression.coef_):
    rhs+=coeff*good_feature_equation
rhs+=linear_regression.intercept_

omega_hat_regression_eq = sp.Eq(symbols.omega_hat,rhs)
omega_hat_regression_eq

In [ ]:
omega_hat_regression_lambda = lambdify(omega_hat_regression_eq.rhs)

In [ ]:
import dill
dill.settings['recurse'] = True
dill.dump(omega_hat_regression_lambda,open('omega0_hat_polynom.sym', 'wb'))

In [ ]:
omega_hat = run(omega_hat_regression_lambda,X)

In [ ]:
fig,ax=plt.subplots()
ax.plot(model.predict(X2[:,good_index.index]),omega_hat,'o')

In [ ]:
X.head()

In [ ]:
fig,ax = plt.subplots()
ax.plot(y_test,model.predict(X_test),'.', alpha=0.5)
ax.set_title('Prediction of %s' % y_key)
ax.set_xlabel('test: %s' % y_key)
ax.set_ylabel('predicted: %s' % y_key)


In [ ]:
import pickle
filename = 'omega0_hat_model.sav'
model.fit(X=X, y=y)
model.keys=list(X.columns)
pickle.dump(model, open(filename, 'wb'))